In [15]:
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-llms-anthropic llama-index-multi-modal-llms-anthropic
!pip install llama-index-embeddings-huggingface
!pip install llama-parse


  Using cached SQLAlchemy-2.0.35-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------------ --- 1.6/1.7 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/15.5 MB 7.6 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/15.5 MB 8.0 MB/s eta 0:00:02
   ------------- -------------------------- 5.2/15.5 MB 8.6 MB/s eta 0:00:02
   ------------------ --------------------- 7.3/15.5 MB 9.2 MB/s eta 0:00:01
   ---------------------- --------

  You can safely remove it manually.
  You can safely remove it manually.


   ---------------------------------------- 0.0/862.7 kB ? eta -:--:--
   ---------------------------------------- 862.7/862.7 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: anthropic
    Found existing installation: anthropic 0.34.2
    Uninstalling anthropic-0.34.2:
      Successfully uninstalled anthropic-0.34.2
   ---------------------------------------- 0.0/762.2 kB ? eta -:--:--
   --------------------------------------- 762.2/762.2 kB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/199.4 MB 12.2 MB/s eta 0:00:17
    --------------------------------------- 4.5/199.4 MB 11.2 MB/s eta 0:00:18
   - -------------------------------------- 6.6/199.4 MB 10.9 MB/s eta 0:00:18
   - -------------------------------------- 8.9/199.4 MB 10.9 MB/s eta 0:00:18
   -- ------------------------------------- 10.7/199.4 MB 10.5 MB/s eta 0:00:18
   -- ------------------------------------- 12.1/199.4 MB 10.1 M

In [3]:
""" This code uses OpenAI's multimodal model gpt-4o-mini for image understanding and generating final RAG response. For creating text and image
embeddings, Llamaindex's embedding model is used."""
import nest_asyncio
from llama_index.core.schema import ImageDocument
from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings
from llama_parse import LlamaParse
import os
from llama_index.core.schema import TextNode
from typing import List
from typing import List
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import openai
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings
from llama_parse import LlamaParse

from llama_index.core.schema import TextNode
from typing import List
import base64
from llama_index.core.schema import ImageDocument
from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal
from llama_index.core import VectorStoreIndex


nest_asyncio.apply()
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = ""

###OpenAI model
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_text_nodes(json_list: List[dict]):
    text_nodes = []
    for idx, page in enumerate(json_list):
        text_node = TextNode(text=page["text"], metadata={"page": page["page"]})
        text_nodes.append(text_node)
    return text_nodes
def encode_image(image_path):
    """Helper function to encode an image as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_image_text_nodes(json_objs: List[dict]):
    from openai import OpenAI
    """Extract out text from images using GPT-4o-mini, with error handling for failed image processing."""
    image_dicts = parser.get_images(json_objs, download_path="images")
    img_text_nodes = []
    client = OpenAI(api_key='')
    print("Understanding the extracted images...")
    for image_dict in image_dicts:
        try:
            # Encode the image as base64
            base64_image = encode_image(image_dict["path"])
            
            # Create the message for GPT-4o-mini
            messages = [
                {
                    "role": "system",
                    "content": "You are an assistant that describes images as alt text."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Describe this image."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ]
            
            # Use GPT-4o-mini to describe the image
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=0.0
            )
            
            # Extract the response and create a text node
            image_description = response.choices[0].message.content
            text_node = TextNode(text=image_description, metadata={"path": image_dict["path"]})
            img_text_nodes.append(text_node)
        
        except Exception as e:
            # Handle the error and append a fallback message
            print(f"Error processing image {image_dict['path']}: {str(e)}")
            img_text_nodes.append(TextNode(text="Could not understand the image", metadata={"path": image_dict["path"]}))
    
    return img_text_nodes

llm  = OpenAI(temperature=0, model="gpt-4o-mini", api_key="")
Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"
parser = LlamaParse(verbose=True)
json_objs = parser.get_json_result("./annual-report-2022.pdf")
json_list = json_objs[0]["pages"]
print("Extracting text nodes.")
text_nodes = get_text_nodes(json_list)
!mkdir llama2_images
print("Extracting text-image nodes...")
image_text_nodes = get_image_text_nodes(json_objs)
print("Parsing complete. Creating index...")
index = VectorStoreIndex(text_nodes + image_text_nodes)
query_engine = index.as_query_engine()
print("All done.")


Started parsing the file under job_id d7890ada-7641-4987-8c76-52e6e16f4f95
Extracting text nodes.
Extracting text-image nodes...
> Image for page 1: [{'name': 'img_p0_1.png', 'height': 231, 'width': 347, 'x': 206.1499023, 'y': 35.39991949999995, 'original_width': 561, 'original_height': 374}, {'name': 'img_p0_2.png', 'height': 515, 'width': 824, 'x': 80.89991, 'y': 355.69941889999996, 'original_width': 1339, 'original_height': 837}]
> Image for page 2: []
> Image for page 3: []
> Image for page 4: []
> Image for page 5: [{'name': 'img_p4_1.png', 'height': 719, 'width': 916, 'x': 56.6999168, 'y': 312.7329272, 'original_width': 900, 'original_height': 706}]
> Image for page 6: []
> Image for page 7: []
> Image for page 8: []
> Image for page 9: []
> Image for page 10: []
> Image for page 11: []
> Image for page 12: []
> Image for page 13: [{'name': 'img_p12_1.png', 'height': 939, 'width': 916, 'x': 56.6999168, 'y': 125.70558349999999, 'original_width': 900, 'original_height': 922}]
> Ima

In [4]:
# ask question
response = query_engine.query(
    "Explain the statistics mentioned in the table Haaga-Helia’s programmes (education that began during 2022, 2021 or 2020)"
)
print(str(response))

The statistics in the table for Haaga-Helia's educational programs provide insights into the performance and trends of the Bachelor's Degree Programmes (AMK), Master's Degree Programmes (YAMK), and the School of Vocational Teacher Education over the years 2020, 2021, and 2022. 

For the Bachelor's Degree Programmes, the appeal, measured as the ratio of primary applicants to initial intake, shows a decline from 3.4 in 2020 to 2.7 in 2022, indicating a decrease in the attractiveness of these programs relative to the number of students they can accept. The number of primary applicants decreased from 10,069 in 2020 to 8,579 in 2022, while the initial intake increased slightly from 2,938 to 3,210, suggesting a more competitive selection process. The total number of students enrolled also decreased, from 9,697 in 2020 to 8,709 in 2022, but the degrees awarded saw a slight increase in 2022 compared to 2020.

In the Master's Degree Programmes, the appeal also decreased from 3.3 in 2020 to 1.7 